In [ ]:
from dotenv import load_dotenv
load_dotenv()

Add a default value of eight hours to the tool function.

In [ ]:
from langchain_anthropic import ChatAnthropic
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver
from datetime import date

def get_today():
    return date.today().isoformat()

def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

@tool
def book_time_entry(project: str, date, hours: int = 8):
    """
    Book time entry for a project.
    Args:
        project (str): The project name.
        date (str): The date for the time entry.
        hours (int): The number of hours to book.
    """
    return f"Booked time entry for {project}."

# Graph
builder = StateGraph(MessagesState)

# LLM with tools
llm = ChatAnthropic(model="claude-3-7-sonnet-20250219")
tools = [book_time_entry]
llm_with_tools = llm.bind_tools(tools)

# System message
sys_msg = SystemMessage(
    content=f"""
    You are a helpful assistant tasked with helping Intertech employees track time spent on projects.
    Today is {get_today()}.
    """)

# Nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

# Show
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "1"}}

messages = [HumanMessage(content="Book pto for yesterday")]
messages = graph.invoke({"messages": messages}, config)

for m in messages['messages']:
    m.pretty_print()